In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [10]:
pc1 = torch.tensor([[[1,2,3],[4,5,6]]])
pc2 = torch.tensor([[[3,2,1],[6,5,4],[9,8,7]]])

点云标准化 pc_noramlization

In [26]:
pc_normal = pc1[0].numpy()
print(pc_normal)

[[1 2 3]
 [4 5 6]]


In [33]:
l = pc_normal.shape[0]
center = np.mean(pc_normal, axis=0)
print(center)
pc = pc_normal - center

[2.5 3.5 4.5]


In [34]:
np.sum(pc**2,axis=1)


array([6.75, 6.75])

In [35]:
np.sqrt(np.sum(pc**2,axis=1))

array([2.59807621, 2.59807621])

In [37]:
m = np.max(np.sqrt(np.sum(pc**2,axis=1)))

In [39]:
pc = pc / m 
print(pc)

[[-0.22222222 -0.22222222 -0.22222222]
 [ 0.22222222  0.22222222  0.22222222]]


两个点云之间的距离计算  square_distance

In [ ]:
pc1 = torch.tensor([[[1,2,3],[4,5,6]]])
pc2 = torch.tensor([[[3,2,1],[6,5,4],[9,8,7]]])

In [57]:
def square_distance_show(src, dst):
    #这里计算的是原点（src）集合中N个点到目标点（dst）集合中M点的距离（平方距离，没有开根号），以Batch为单位，输出B×N×M的tensor。
    """
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape#单下划线表示不关心
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1)) #permute为转置,[B, N, M]
    print(dist)
    #torch.matmul()也是一种类似于矩阵相乘操作的tensor联乘操作。但是它可以利用python 中的广播机制,处理一些维度不同的tensor结构进行 相乘操作。
    dist += torch.sum(src ** 2, -1).view(B, N, 1)#[B, N, M] + [B, N, 1]，dist的每一列都加上后面的列值
    print('torch.sum(src ** 2, -1)',torch.sum(src ** 2, -1))
    print('torch.sum(src ** 2, -1).shape',torch.sum(src ** 2, -1).shape)
    print('torch.sum(src ** 2, -1).view(B, N, 1)',torch.sum(src ** 2, -1).view(B, N, 1))
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)#[B, N, M] + [B, 1, M],dist的每一行都加上后面的行值
    print(torch.sum(dst ** 2, -1).view(B, 1, M))
    A = torch.sum(src ** 2, -1).view(B, N, 1)+torch.sum(dst ** 2, -1).view(B, 1, M)
    print('A',A)
    print('RESUlt',dist)
    return dist

In [56]:
# N = 2  M = 3 
square_distance_show(pc1,pc2)
# 相当于横行的都是某一个A1的x，y，z的平方和
# 纵行都是某一个B1的x，y，z的平方和
# 所以可以让他们通过维度扩展

tensor([[[ -20,  -56,  -92],
         [ -56, -146, -236]]])
torch.sum(src ** 2, -1) tensor([[14, 77]])
torch.sum(src ** 2, -1).shape torch.Size([1, 2])
torch.sum(src ** 2, -1).view(B, N, 1) tensor([[[14],
         [77]]])
tensor([[[ 14,  77, 194]]])
A tensor([[[ 28,  91, 208],
         [ 91, 154, 271]]])
RESU tensor([[[  8,  35, 116],
         [ 35,   8,  35]]])


tensor([[[  8,  35, 116],
         [ 35,   8,  35]]])

In [71]:
pc1 = torch.tensor([[[1,2,3],[4,5,6]]])
sum = torch.sum(pc1**2,dim=-1)
print(sum)
print(sum.view(1,2,1))
print(sum.view(1,2,1).repeat(1,1,3))
pc2 = torch.tensor([[[3,2,1],[6,5,4],[9,8,7]]])
sum1 = torch.sum(pc2**2,dim=-1)
print(sum1.view(1,1,3).repeat(1,2,1))
print(sum1.view(1,1,3).repeat(1,2,1)+sum.view(1,2,1).repeat(1,1,3))

tensor([[14, 77]])
tensor([[[14],
         [77]]])
tensor([[[14, 14, 14],
         [77, 77, 77]]])
tensor([[[ 14,  77, 194],
         [ 14,  77, 194]]])
tensor([[[ 28,  91, 208],
         [ 91, 154, 271]]])


index_points 
 [B,N,C] [B,S] -> [B,S,C]

In [129]:
def index_points(points,idx):
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1) # len(view_shape)固定为2，所以就是改成->[B,1]
    repeat_shape = list(idx.shape) # [B,S]
    repeat_shape[0] = 1 # [1,S]
    batch_indices = torch.arange(B, dtype=torch.long).view(view_shape).repeat(repeat_shape)# 改成[B,S]，其中代表的是每个地方的索引[[0,0],[1,1]]
    print('idx',idx)
    print('batch_indicies',batch_indices)
    new_points = points[batch_indices, idx, :]
    # 这个切片操作，batch_indices和idx的个数一定要匹配
    print(points[:, idx, :])
    return new_points
'''
如果是points[:, idx, :],那么在这个例子中两个Batch的[0,2]都会被取到,idx相当于取了八次,idx里面有4个数,4*b=8
如果是points[batch_dict, idx, :]那么就是取了4次,因为前面指定了每个b
'''


In [130]:
pc1 = torch.tensor([[[1,2,3],[4,5,6],[7,8,9]],[[2,2,3],[5,5,6],[8,8,9]]])
idx = torch.tensor([[0,2],[1,2]])
print(pc1.shape)
print(idx.shape)

torch.Size([2, 3, 3])
torch.Size([2, 2])


In [132]:
pc1 = torch.tensor([
                    [[1,2,3],[4,5,6],[7,8,9]]
                    ,[[2,2,3],[5,5,6],[8,8,9]]
                    ])
idx = torch.tensor([[0,2],[1,2]])

In [155]:
pc1  = [[[1,2,3],[4,5,6],[7,8,9]]
        ,[[2,2,3],[5,5,6],[8,8,9]]]
pc1 = np.array(pc1)
print(pc1.shape)

(2, 3, 3)


In [156]:
pc1[0][0][0]

1

In [157]:
pc1

array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[2, 2, 3],
        [5, 5, 6],
        [8, 8, 9]]])

In [159]:
pc1[0,0,0]

1

In [160]:
pc1[:,[0,2],:]

array([[[1, 2, 3],
        [7, 8, 9]],

       [[2, 2, 3],
        [8, 8, 9]]])

In [168]:
pc1[:,[[0,2],[1,2]],:]

array([[[[1, 2, 3],
         [7, 8, 9]],

        [[4, 5, 6],
         [7, 8, 9]]],


       [[[2, 2, 3],
         [8, 8, 9]],

        [[5, 5, 6],
         [8, 8, 9]]]])

In [169]:
pc1[:,[[0,2],[1,2,3]],:]

/tmp/ipykernel_336/3192601660.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pc1[:,[[0,2],[1,2,3]],:]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [176]:
pc1[[0,1],[[0,2],[1,2]]]

array([[[1, 2, 3],
        [8, 8, 9]],

       [[4, 5, 6],
        [8, 8, 9]]])

In [180]:
pc1[[0,0],[[0,2],[1,2]]]

array([[[1, 2, 3],
        [7, 8, 9]],

       [[4, 5, 6],
        [7, 8, 9]]])

In [181]:
pc1[[[0],[1]],[[0,2],[1,2]]]

array([[[1, 2, 3],
        [7, 8, 9]],

       [[5, 5, 6],
        [8, 8, 9]]])

In [182]:
pc1[[[0],[1]],[0,2]]

array([[[1, 2, 3],
        [7, 8, 9]],

       [[2, 2, 3],
        [8, 8, 9]]])

In [213]:
def index_points_gather(points, idx):
    B, N, C = points.shape
    _, S = idx.shape
    print(idx)
    idx = idx.view(B, S, 1)
    print(idx)
    idx = idx.repeat(1, 1, C)
    print(idx)
    new_points = torch.gather(points, dim=1, index=idx)
    print(points)
    return new_points

In [210]:
pc1 = torch.tensor([[[1,2,3],[4,5,6],[7,8,9]],[[2,2,3],[5,5,6],[8,8,9]]])
idx = torch.tensor([[0,2],[1,2]])

In [214]:
index_points_gather(pc1,idx)

tensor([[0, 2],
        [1, 2]])
tensor([[[0],
         [2]],

        [[1],
         [2]]])
tensor([[[0, 0, 0],
         [2, 2, 2]],

        [[1, 1, 1],
         [2, 2, 2]]])
tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]],

        [[2, 2, 3],
         [5, 5, 6],
         [8, 8, 9]]])


tensor([[[1, 2, 3],
         [7, 8, 9]],

        [[5, 5, 6],
         [8, 8, 9]]])

In [220]:
idx_manul = [[0,2],[0,2],[0,2]]

In [221]:
n1 = torch.gather(pc1,dim=0,index=idx_manul)

TypeError: gather() received an invalid combination of arguments - got (Tensor, index=list, dim=int), but expected one of:
 * (Tensor input, int dim, Tensor index, *, bool sparse_grad, Tensor out)
 * (Tensor input, name dim, Tensor index, *, bool sparse_grad, Tensor out)


torch.gather()

In [198]:
b = torch.Tensor([[1,2,3],[4,5,6]])
print(b)
print(b.shape)
index_1 = torch.LongTensor([[0,1],[2,0]])
index_2 = torch.LongTensor([[0,1,1],[0,0,0]])
print (torch.gather(b, dim=1, index=index_1)) # 按照行
print (torch.gather(b, dim=0, index=index_2)) # 按照列

tensor([[1., 2., 3.],
        [4., 5., 6.]])
torch.Size([2, 3])
tensor([[1., 2.],
        [6., 4.]])
tensor([[1., 5., 6.],
        [1., 2., 3.]])


In [203]:
torch.gather(pc1,dim=0,index=[[[1]]])

TypeError: gather() received an invalid combination of arguments - got (Tensor, index=list, dim=int), but expected one of:
 * (Tensor input, int dim, Tensor index, *, bool sparse_grad, Tensor out)
 * (Tensor input, name dim, Tensor index, *, bool sparse_grad, Tensor out)
